
# Теория Кодирования
## Лабораторная работа 1

От студента группы **6405**-010302
**Слеповичева Святослава**


---

### Задание
#### 1.1 Реализовать функцию REF(), приводящую матрицу к ступенчатому виду


In [353]:
# Для математических расчётов и хранения данных о матрицах
import numpy as np

# Для "фокусов" с пунктами 1.4
import itertools

In [354]:
# Для удобства использования и приятного чтения кода,
# в большинстве своём, используется статическое определение типа переменных,
# которые были прописаны заранее

def number_ones(current_column: np.ndarray,
                level_column: int,
                miss_rows: int) -> int | None:
    '''
    Функция, возвращающая индексы столбцов с единицами в нужной строке.\n
    Если в столбце нет единиц, функция возвращает None
    :param current_column: столбец с подозрением на сокрытие единиц
    :param level_column: откуда начать проверку (уровень глубины проверки)
    :param miss_rows: сколько строк уже было пропущено
    :return: индексы столбцов с единицами
    '''
    one_indexes = []

    for level in range(len(current_column)):
        if current_column[level] == 1:
            one_indexes.append(level + level_column - miss_rows)

    if len(one_indexes) == 0:
        return None

    return one_indexes


def REF(x: np.ndarray) -> np.ndarray:
    '''
    Функция приведения матрицы к ступенчатому виду
    :param x: матрица numpy
    :return: ступенчатая матрица
    '''
    no_digit: int = 0

    for col in range(x.shape[1]):
        ones_indexes: (int | None) = number_ones(x[col - no_digit:, col],
                                                 col,
                                                 no_digit)
        if ones_indexes is None:
            no_digit += 1
            continue
        if len(ones_indexes) == 1:
            x[[ones_indexes[0], col - no_digit], :] = x[[col - no_digit, ones_indexes[0]], :]
        elif len(ones_indexes) > 1:
            x[[ones_indexes[0], col - no_digit]] = x[[col - no_digit, ones_indexes[0]]]
            for i in ones_indexes[1:]:
                x[i] += x[col - no_digit]
                for j in range(len(x[col])):
                    if x[i, j] == 2: x[i, j] = 0
    return x

#### 1.2. Реализовать функцию RREF(), приводящую матрицу к приведённому ступенчатому виду.

In [355]:

def first_one(row: np.ndarray) -> int | None:
    '''
    Функция поиска индекса первой пойманной единицы в строке
    :param row: строка для поиска
    :return: индекс пойманной единицы или None, если единиц не обнаружено
    '''
    for i in range(len(row)):
        if row[i] == 1:
            return i
    return None


def RREF(x: np.ndarray) -> np.ndarray:
    '''
    Функция приведения матрицы к приведённому ступенчатому виду
    :param x: матрица numpy
    :return: приведённая ступенчатая матрица
    '''
    # Приводим данную нам матрицу уже знакомым нам методом
    x = REF(x)

    for col in range(x.shape[0]):
        first_one_index: int | None = first_one(x[col])

        if first_one_index is None:
            x = np.delete(x, np.s_[col:], axis=0)
            break

        for i in range(x.shape[0]):
            if (x[i, first_one_index] == 1) and (i != col):
                x[i] += x[col]
                for j in range(len(x[col])):
                    if x[i, j] == 2: x[i, j] = 0
    return x


#### 1.3. Создать класс линейных кодов LinearCode.
##### 1.3.1 На основе входной матрицы сформировать порождающую матрицу в ступенчатом виде.

In [356]:
linear_code = np.array(
    [[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
     [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

matrix_code = REF(linear_code)


# Избавимся от "лишних", полностью нулевых строк
for i in range(matrix_code.shape[0]):
    if sum(matrix_code[i]) == 0:
        matrix_code = np.delete(matrix_code, np.s_[i:], axis=0)
        break

##### 1.3.2 Задать n равное числу столбцов и k равное числу строк полученной матрицы
(без учёта полностью нулевых строк).

In [357]:
k, n = matrix_code.shape

##### 1.3.3 Сформировать проверочную матрицу на основе порождающей.

In [358]:

# Шаг 1. Сформировать матрицу 𝐆∗ в приведённом ступенчатом виде на основе порождающей.
g_matrix = RREF(matrix_code)

# Шаг 2. Зафиксировать ведущие столбцы lead матрицы 𝐆∗.
lead_columns = []
for i in range(k):
    for j in range(n):
        if g_matrix[i, j] == 1:
            lead_columns.append(j)
            break

# Шаг 3. Сформировать сокращённую матрицу 𝐗,
# удалив ведущие столбцы матрицы 𝐆∗.
x_matrix = np.delete(g_matrix, lead_columns, axis=1)

# Шаг 4. Сформировать матрицу 𝐇, поместив в строки,
# соответствующие позициям ведущих столбцов строки из 𝐗,
# а в остальные – строки единичной матрицы.
h_matrix = np.zeros((n, n - k), dtype=int)

x_matrix_col: int = 0
one_matrix_col: int = 0

for i in range(h_matrix.shape[0]):
    if i in lead_columns:
        h_matrix[i] = x_matrix[x_matrix_col].copy()
        x_matrix_col += 1
    else:
        h_matrix[i, one_matrix_col] = 1
        one_matrix_col += 1

print(h_matrix)

[[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


#### 1.4. Сформировать все кодовые слова длины n двумя способами.
##### 1.4.1 Сложить все слова из порождающего множества, оставить неповторяющиеся.

In [359]:

indexes = []
for i in range(2, matrix_code.shape[0] + 1):
    indexes.extend(itertools.combinations(range(matrix_code.shape[0]), i))


key_words = [*np.zeros((1, matrix_code.shape[1]), dtype=int), *matrix_code[:]]
for el in indexes:
    summ = np.zeros((1, matrix_code.shape[1]), dtype=int)

    for i in range(len(el)):
        summ += matrix_code[el[i]]

    for i in range(len(summ)):
        summ[i] %= 2

    key_words.append(*summ)
    print(key_words[-1])



[1 0 1 1 0 0 0 1 0 0 1]
[1 0 1 0 1 1 0 0 0 1 1]
[1 0 1 0 0 1 1 1 0 0 0]
[1 0 1 0 0 1 0 1 1 0 1]
[0 0 0 1 1 1 0 1 0 1 0]
[0 0 0 1 0 1 1 0 0 0 1]
[0 0 0 1 0 1 0 0 1 0 0]
[0 0 0 0 1 0 1 1 0 1 1]
[0 0 0 0 1 0 0 1 1 1 0]
[0 0 0 0 0 0 1 0 1 0 1]
[1 0 1 1 1 0 0 0 0 0 0]
[1 0 1 1 0 0 1 1 0 1 1]
[1 0 1 1 0 0 0 1 1 1 0]
[1 0 1 0 1 1 1 0 0 0 1]
[1 0 1 0 1 1 0 0 1 0 0]
[1 0 1 0 0 1 1 1 1 1 1]
[0 0 0 1 1 1 1 1 0 0 0]
[0 0 0 1 1 1 0 1 1 0 1]
[0 0 0 1 0 1 1 0 1 1 0]
[0 0 0 0 1 0 1 1 1 0 0]
[1 0 1 1 1 0 1 0 0 1 0]
[1 0 1 1 1 0 0 0 1 1 1]
[1 0 1 1 0 0 1 1 1 0 0]
[1 0 1 0 1 1 1 0 1 1 0]
[0 0 0 1 1 1 1 1 1 1 1]
[1 0 1 1 1 0 1 0 1 0 1]


In [360]:
# Список ключевых слов
key_words_tuple = []
for elmt in key_words:
    key_words_tuple.append(tuple(elmt))

# Оставим только уникальные ключевые слова
unic_key_words = set(key_words_tuple)

unic_key_words

{(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1),
 (0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1),
 (0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0),
 (0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1),
 (0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0),
 (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1),
 (0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0),
 (0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0),
 (0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1),
 (0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0),
 (0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1),
 (1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0),
 (1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1),
 (1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0),
 (1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1),
 (1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1),
 (1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0),
 (1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1),
 (1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0),
 (1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1),
 (1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0),
 (1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1),
 (1, 0, 1, 1, 0, 0, 1, 1, 1,

##### 1.4.2 Взять все двоичные слова длины k, умножить каждое на G.

In [361]:

all_key_words = itertools.product(range(2), repeat=k)

all_key_words_res = []
for elmt in all_key_words:
    result = np.array(elmt).dot(matrix_code)
    for i in range(len(result)):
        result[i] %= 2
    all_key_words_res.append(result)


all_key_words_result = []
for elmt in all_key_words_res:
    all_key_words_result.append(tuple(elmt))
all_key_words_result = set(all_key_words_result)

# Проверка на совпадение списков слов
print(unic_key_words == all_key_words_result)


True


In [362]:
# Проверка нулевых векторов при перемножении
mult = [elmt.dot(h_matrix) for elmt in key_words]

for elmt in mult:
    for i in range(len(elmt)): elmt[i] %= 2


all([(elmt == np.zeros((1, n - k), dtype=int)).all() for elmt in mult])

True

#### 1.4 Вычислить кодовое расстояние получившегося кода.

In [363]:
# То есть найти наименьшую длину кодового слова, при этом не равного нулю.
d = n
for elmt in all_key_words_result:
    summ = sum(elmt)
    if (summ < d) and (summ != 0):
        d = summ

print(f"Code distance = {d}")

Code distance = 2


##### 1.4.1 Внести в кодовое слово ошибку кратности не более t, умножить полученное слово на H, убедиться в обнаружении ошибки.

In [364]:
# t - error - ошибка
t = d - 1

print(f"Detected error's multiplicity: {t}")

Detected error's multiplicity: 1


In [365]:
for elmt in key_words:
    for i in range(len(elmt)):
        err = elmt.copy()
        err[i] = 1 if err[i] == 0 else 0

        if sum(err.dot(h_matrix)) != 0:
            print(f"element = {elmt}")
            print(f"error = {err}")
            print(f"error mult h_matrix = {err.dot(h_matrix)}")
            break
        else:
            continue
    break

element = [0 0 0 0 0 0 0 0 0 0 0]
error = [1 0 0 0 0 0 0 0 0 0 0]
error mult h_matrix = [0 1 1 1 1 0]


##### 1.4.2 Найти для некоторого кодового слова ошибку кратности t+1 такую, что при умножении на H ошибка не может быть обнаружена.

In [366]:
for elmt in key_words:
    for i in range(len(elmt)):
        for j in range(len(elmt)):
            err = elmt.copy()

            if i != j:
                err[i] = 1 if err[i] == 0 else 0

                err[j] = 1 if err[j] == 0 else 0

                if sum(err.dot(h_matrix)) == 0:
                    print(f"el: {elmt}")
                    print(f"error: {err}")
                    print(f"error * h_matrix: {err.dot(h_matrix)}")
                    break
        else:
            continue
        break
    else:
        continue
    break

el: [0 0 0 0 0 0 1 0 0 1 0]
error: [0 0 0 0 0 0 0 0 0 0 0]
error * h_matrix: [0 0 0 0 0 0]


Объяснение "невидимости":
1) Ошибки будут видны только те, которые меньше дистанции на один (d - 1),
2) Если H (h_matrix) - проверочная матрица линейного кода C (linear_code) длины n,
тогда C состоит из всех слов $v ∈ K_n$ таких, что $v$H=0.
Тогда если err - слово с ошибкой кратности t и $err⋅H≠0$,
тогда err обнаружится, иначе - нет.